In [ ]:
from ragas.testset import TestsetGenerator
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings

from langchain.chat_models import ChatOpenAI

from ragas.llms import LangchainLLM

# Create Test set

In [ ]:
loader = PyPDFLoader("../data/Batman_wiki.pdf")
pages = loader.load()

In [ ]:
# Add custom llms and embeddings
generator_llm = LangchainLLM(ChatOpenAI(model="gpt-3.5-turbo"))
critic_llm = LangchainLLM(ChatOpenAI(model="gpt-4"))
embeddings_model = OpenAIEmbeddings()

# Change resulting question type distribution
testset_distribution = {
    "simple": 1.0,
    "reasoning": 0.0,
    "multi_context": 0.0,
    "conditional": 0.0,
}

# percentage of conversational question
chat_qa = 0

testset_generator = TestsetGenerator(
    generator_llm,
    critic_llm,
    embeddings_model,
    testset_distribution=testset_distribution,
    chat_qa=chat_qa,
)

evalset = testset_generator.generate(pages, test_size=10)
evalset_df = evalset.to_pandas()
evalset_df.head()

/Users/chan.k.20/.local/lib/python3.11/site-packages/ragas/testset/testset_generator.py:314: UserWarning: Most documents are too short
  warnings.warn("Most documents are too short")
  0%|          | 0/10 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# testsetgenerator = TestsetGenerator.from_default()
# test_size = 2
# testset = testsetgenerator.generate(pages, test_size=test_size)
# test_df = testset.to_pandas()
# test_df.head()

3it [03:25, 68.50s/it]


,question,context,answer,question_type,episode_done
0,What abilities and skills does Batman have and...,- Batman has no inherent superhuman powers; he...,Batman has a wide range of abilities and skill...,multi_context,True
1,"What character traits define Batman, consideri...",- Bruce Wayne's vast personal wealth is import...,The character traits that define Batman includ...,multi_context,True


In [ ]:
evalset_df['question_type'].value_counts()

question_type
simple    10
Name: count, dtype: int64

# Save test set

In [ ]:
# test_df.to_csv("../data/batman_eval_simple.csv", index=False)